In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import json

import sys 
sys.path.append('..')

from utilities import health_data
from utilities import configuration

config = configuration.get_config()

In [2]:
experiment_configurations = json.load(open(config['experiments_config'], encoding='utf-8'))
X_train, y_train, X_test, y_test, columns = health_data.Admission.get_train_test_matrices(experiment_configurations['configuration_15'])

/home/maiso/cbu/environments/pytorch/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


2024-01-12 16:21:50,502 - root - DEBUG - Loading diagnosis embeddings from model: diag_conf_1
Computing diagnosis embeddings ...
2024-01-12 16:21:53,612 - root - DEBUG - Embedding loaded, shape=(419139, 100)
2024-01-12 16:21:55,332 - root - DEBUG - Loading intervention embeddings from model: interv_conf_1
Precomputed embeddings found 524023
2024-01-12 16:21:56,951 - gensim.utils - INFO - loading Doc2Vec object from /home/maiso/cbu/repositories/alternate-level-of-care-project/gensim/models/interv_conf_1
2024-01-12 16:21:56,952 - smart_open.smart_open_lib - DEBUG - {'uri': '/home/maiso/cbu/repositories/alternate-level-of-care-project/gensim/models/interv_conf_1', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
2024-01-12 16:21:57,086 - gensim.utils - INFO - loading dv recursively from /home/maiso/cbu/repositories/alternate-level-of-care-project/gensim/models

In [9]:
reduced=5000
X = torch.tensor(X_train[:reduced,:].toarray(), dtype=torch.float32)
y = torch.tensor(np.vstack([y_train[:reduced]==0, y_train[:reduced]==1]).T, dtype=torch.float32)

In [26]:
feature_count=X.shape[1]
components = []
hidden_layers = [100,]
last_layer_size = feature_count


for ix, size_layer in enumerate(hidden_layers):
    components.append(nn.Linear(last_layer_size, size_layer))
    components.append(nn.ReLU())
    last_layer_size=size_layer


components.append(nn.Linear(last_layer_size, 2))

components.append(nn.Softmax(dim=1))

model = nn.Sequential(*components)
print(model)

Sequential(
  (0): Linear(in_features=273, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=2, bias=True)
  (3): Softmax(dim=1)
)


In [27]:
weights = torch.FloatTensor([0.5, 12]) 
loss_fn = nn.BCELoss(weight=weights)  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [44]:
n_epochs = 50
batch_size=100

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 1.2524518966674805
Finished epoch 1, latest loss 1.2472009658813477
Finished epoch 2, latest loss 1.2473106384277344
Finished epoch 3, latest loss 1.2485438585281372
Finished epoch 4, latest loss 1.2483597993850708
Finished epoch 5, latest loss 1.24614679813385
Finished epoch 6, latest loss 1.2406388521194458
Finished epoch 7, latest loss 1.2435001134872437
Finished epoch 8, latest loss 1.2497715950012207
Finished epoch 9, latest loss 1.2585740089416504
Finished epoch 10, latest loss 1.2542248964309692
Finished epoch 11, latest loss 1.2533665895462036
Finished epoch 12, latest loss 1.2586696147918701
Finished epoch 13, latest loss 1.264222264289856
Finished epoch 14, latest loss 1.2637180089950562
Finished epoch 15, latest loss 1.2837531566619873
Finished epoch 16, latest loss 1.2687216997146606
Finished epoch 17, latest loss 1.2876534461975098
Finished epoch 18, latest loss 1.2922255992889404
Finished epoch 19, latest loss 1.2867072820663452
Finished epoc

In [48]:
np.sum(y_pred)

0.0

In [45]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_score = model(X)[:,1]
    
y_pred = y_score.round().numpy()
y_score = y_score.numpy()
y_true = y[:,1].numpy()


print(f'precision={precision_score(y_true, y_pred):5.4f}')
print(f'recall=   {recall_score(y_true, y_pred):5.4f}')
print(f'f1_score= {f1_score(y_true, y_pred):5.4f}')
print(f'AUC= {roc_auc_score(y_true=y_true, y_score=y_score):5.4f}')

precision=0.0000
recall=   0.0000
f1_score= 0.0000
f1_score= 0.7846


/home/maiso/cbu/environments/pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
                    f1_score(y_true, y_pred,),
                    roc_auc_score(y_true=y_true, y_score=y_pred),

In [33]:
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)

accuracy = (y_pred[:,1].round() == y[:,1]).float().mean()


print(f"Accuracy {accuracy}")

Accuracy 0.9350000023841858


In [ ]:


# make probability predictions with the model
predictions = model(X)
# round predictions
rounded = predictions.round()

In [ ]:

# make class predictions with the model
predictions = (model(X) > 0.5).int()